In [1]:
# rename_umbrella.py
import glob, os
from tqdm import tqdm

LBL_TRAIN = r"C:\coco_subset\labels\train"
LBL_VAL   = r"C:\coco_subset\labels\val"

for lbl in tqdm(glob.glob(os.path.join(LBL_TRAIN, "*.txt"))
                + glob.glob(os.path.join(LBL_VAL, "*.txt"))):
    lines = []
    for ln in open(lbl):
        parts = ln.split()
        if parts[0] == "1":        # class 1 = umbrella → umbrella_opened
            parts[0] = "1"         # 숫자는 그대로, names 만 바꿀 예정
        lines.append(" ".join(parts)+"\n")
    open(lbl, "w").writelines(lines)

print("✓ umbrella ⇒ umbrella_opened 이름 매핑 완료")


100%|██████████| 8991/8991 [00:14<00:00, 623.93it/s]

✓ umbrella ⇒ umbrella_opened 이름 매핑 완료


In [2]:
# extract_closed_from_train2017.py
import os, shutil, cv2, numpy as np
from pathlib import Path
from pycocotools.coco import COCO

COCO_IMG   = r"C:\coco\train2017"
COCO_ANN   = r"C:\coco\annotations\instances_train2017.json"
OUT_IMG    = r"C:\coco_subset\images\train"   # train 세트에 합류
OUT_LBL    = r"C:\coco_subset\labels\train"
AR_TH      = 2.5          # h/w ≥ 2.5 → 접힌 우산
MIN_AREA   = 32*32

coco = COCO(COCO_ANN)
cat = coco.getCatIds(catNms=['umbrella'])[0]

for ann in coco.loadAnns(coco.getAnnIds(catIds=[cat], iscrowd=False)):
    x,y,w,h = ann['bbox']
    if w*h < MIN_AREA or h/w < AR_TH:     # 펴진 우산 skip
        continue

    img_info = coco.loadImgs(ann['image_id'])[0]
    src = os.path.join(COCO_IMG, img_info['file_name'])
    dst = os.path.join(OUT_IMG,  img_info['file_name'])
    shutil.copy(src, dst)

    # YOLO txt (class 2 = umbrella_closed)
    xc,yc = x+w/2, y+h/2
    yolo = f"2 {xc/img_info['width']:.6f} {yc/img_info['height']:.6f} " \
           f"{w/img_info['width']:.6f} {h/img_info['height']:.6f}\n"
    open(os.path.join(
        OUT_LBL, Path(img_info['file_name']).with_suffix(".txt").name),
        "a").write(yolo)          # a모드: 이미지에 opened 박스가 이미 있을 수도 있음
print("✓ umbrella_closed 원본 추출 완료")


loading annotations into memory...
Done (t=9.89s)
creating index...
index created!
✓ umbrella_closed 원본 추출 완료


In [3]:
# augment_closed_v2.py
import os, glob, cv2, numpy as np
from pathlib import Path
from tqdm import tqdm
import albumentations as A

ROOT   = r"C:\coco_subset"
IMGDIR = os.path.join(ROOT, "images", "train")
LBLDIR = os.path.join(ROOT, "labels", "train")
FACTOR = 10

# 1. closed 이미지 목록 ---------------------------------------------------
closed_imgs = [Path(t).with_suffix(".jpg").name
               for t in glob.glob(os.path.join(LBLDIR, "*.txt"))
               if any(l.startswith("2 ") for l in open(t))]
print("closed 원본:", len(closed_imgs))

# 2. 증강 파이프라인 (Albumentations 2.x) ---------------------------------
transform = A.Compose(
    [
        A.HorizontalFlip(p=0.5),
        A.VerticalFlip(p=0.3),
        A.RandomRotate90(p=0.5),
        A.Affine(scale=(0.8, 1.2),
                 translate_percent=(0, 0.1),
                 rotate=(-20, 20),
                 border_mode=cv2.BORDER_REFLECT_101,
                 p=0.7),
        A.RandomBrightnessContrast(0.2, 0.2, p=0.5),
        A.RGBShift(15, 15, 15, p=0.3),
        A.MotionBlur(blur_limit=5, p=0.2),
    ],
    bbox_params=A.BboxParams(
        format="yolo",
        label_fields=["labels"],   # 필수
        min_visibility=0.01        # 1 % 이하로 남으면 버림
        # 2.x에서는 validate / check_each_transform 옵션이 사라졌습니다
    ),
)

# 3. bbox 클램프 함수 ------------------------------------------------------
def sanitize(bboxes):
    fixed = []
    for xc, yc, w, h in bboxes:
        w  = np.clip(w, 1e-6, 1.0)
        h  = np.clip(h, 1e-6, 1.0)
        xc = np.clip(xc, w / 2, 1.0 - w / 2)
        yc = np.clip(yc, h / 2, 1.0 - h / 2)
        fixed.append([xc, yc, w, h])
    return fixed

# 4. 증식 루프 ------------------------------------------------------------
for img_name in tqdm(closed_imgs, desc="augment"):
    img_path = os.path.join(IMGDIR, img_name)
    lbl_path = os.path.join(LBLDIR, Path(img_name).with_suffix(".txt").name)

    image = cv2.imread(img_path)
    if image is None:
        continue

    base_bb = [list(map(float, ln.split()[1:]))
               for ln in open(lbl_path) if ln.startswith("2 ")]
    base_bb = sanitize(base_bb)
    if not base_bb:
        continue
    labels  = ["2"] * len(base_bb)

    # n-1 번 증강
    for k in range(FACTOR - 1):
        aug = transform(image=image, bboxes=base_bb, labels=labels)
        bbs = sanitize(aug["bboxes"])
        if not bbs:
            continue

        stem = f"{Path(img_name).stem}_caug{k}"
        cv2.imwrite(os.path.join(IMGDIR, f"{stem}.jpg"), aug["image"])
        with open(os.path.join(LBLDIR, f"{stem}.txt"), "w") as f:
            for b in bbs:
                f.write("2 " + " ".join(f"{v:.6f}" for v in b) + "\n")

print("✓ closed 10배 증식 완료")


closed 원본: 240


augment: 100%|██████████| 240/240 [00:11<00:00, 20.07it/s]

✓ closed 10배 증식 완료


In [4]:
import glob, os
closed = opened = 0
for t in glob.glob(os.path.join(LBLDIR, "*.txt")):
    for l in open(t):
        if l.startswith("2 "): closed += 1
        elif l.startswith("1 "): opened += 1
print("closed", closed, "opened", opened)

closed 5082 opened 8960


In [5]:
# relabel_closed_to_2.py
import glob, os
from pathlib import Path

ROOT = r"C:\coco_plusdata"
LBL_DIR = os.path.join(ROOT, "labels")

count = 0
for txt in glob.glob(os.path.join(LBL_DIR, "*.txt")):
    changed = []
    for ln in open(txt):
        parts = ln.strip().split()
        if parts and parts[0] == "0":        # 0 → 2
            parts[0] = "2"
            count += 1
        changed.append(" ".join(parts) + "\n")
    Path(txt).write_text("".join(changed))

print(f"라벨 변환 완료: {count:,} 줄 수정")


라벨 변환 완료: 15 줄 수정


In [6]:
# augment_closed_plusdata.py  (Albumentations 2.0.6)
import os, glob, cv2, numpy as np
from pathlib import Path
from tqdm import tqdm
import albumentations as A

ROOT   = r"C:\coco_plusdata"
IMGDIR = os.path.join(ROOT, "images")
LBLDIR = os.path.join(ROOT, "labels")
FACTOR = 10

# closed 이미지 목록(class 2)
closed_imgs = [Path(t).with_suffix(".jpg").name
               for t in glob.glob(os.path.join(LBLDIR, "*.txt"))
               if any(l.startswith("2 ") for l in open(t))]
print("closed 원본:", len(closed_imgs))

transform = A.Compose(
    [
        A.HorizontalFlip(p=0.5),
        A.VerticalFlip(p=0.3),
        A.RandomRotate90(p=0.5),
        A.Affine(scale=(0.8, 1.2), translate_percent=(0, 0.1),
                 rotate=(-20, 20), border_mode=cv2.BORDER_REFLECT_101, p=0.7),
        A.RandomBrightnessContrast(0.2, 0.2, p=0.5),
        A.RGBShift(15, 15, 15, p=0.3),
        A.MotionBlur(blur_limit=5, p=0.2),
    ],
    bbox_params=A.BboxParams(
        format="yolo",
        label_fields=["labels"],
        min_visibility=0.01,
    ),
)

def sanitize(bboxes):
    fixed = []
    for xc, yc, w, h in bboxes:
        w  = np.clip(w, 1e-6, 1.0)
        h  = np.clip(h, 1e-6, 1.0)
        xc = np.clip(xc, w / 2, 1.0 - w / 2)
        yc = np.clip(yc, h / 2, 1.0 - h / 2)
        fixed.append([xc, yc, w, h])
    return fixed

for img_name in tqdm(closed_imgs, desc="augment"):
    img_path = os.path.join(IMGDIR, img_name)
    lbl_path = os.path.join(LBLDIR, Path(img_name).with_suffix(".txt").name)

    image = cv2.imread(img_path)
    if image is None:
        continue

    base_bb = [list(map(float, ln.split()[1:]))
               for ln in open(lbl_path) if ln.startswith("2 ")]
    base_bb = sanitize(base_bb)
    if not base_bb:
        continue
    labels = ["2"] * len(base_bb)

    for k in range(FACTOR - 1):            # 원본 제외 9장
        aug = transform(image=image, bboxes=base_bb, labels=labels)
        bbs = sanitize(aug["bboxes"])
        if not bbs:
            continue

        stem = f"{Path(img_name).stem}_caug{k}"
        cv2.imwrite(os.path.join(IMGDIR, f"{stem}.jpg"), aug["image"])
        with open(os.path.join(LBLDIR, f"{stem}.txt"), "w") as f:
            for b in bbs:
                f.write("2 " + " ".join(f"{v:.6f}" for v in b) + "\n")

print("✓ closed 10배 증식 완료")


closed 원본: 15


augment: 100%|██████████| 15/15 [00:01<00:00,  7.72it/s]

✓ closed 10배 증식 완료


In [2]:
# split_80_20.py
import os, glob, random, shutil
from pathlib import Path

ROOT      = r"C:\coco_plusdata"
IMG_SRC   = os.path.join(ROOT, "images")
LBL_SRC   = os.path.join(ROOT, "labels")

# 새 폴더
for sub in ("images/train", "images/val", "labels/train", "labels/val"):
    os.makedirs(os.path.join(ROOT, sub), exist_ok=True)

# 모든 이미지 목록
imgs = glob.glob(os.path.join(IMG_SRC, "*.*g"))   # jpg, png …
random.shuffle(imgs)

split_idx = int(len(imgs) * 0.8)     # 8 : 2
train_imgs = imgs[:split_idx]
val_imgs   = imgs[split_idx:]

def move_pair(img_path, subset):
    """이미지/라벨 쌍을 train 또는 val 하위로 이동"""
    stem = Path(img_path).stem
    lbl_path = os.path.join(LBL_SRC, f"{stem}.txt")
    dst_img  = os.path.join(ROOT, f"images/{subset}", Path(img_path).name)
    dst_lbl  = os.path.join(ROOT, f"labels/{subset}", f"{stem}.txt")
    shutil.move(img_path, dst_img)
    if os.path.exists(lbl_path):
        shutil.move(lbl_path, dst_lbl)

for p in train_imgs: move_pair(p, "train")
for p in val_imgs:   move_pair(p, "val")

print(f"train : {len(train_imgs):>3}  |  val : {len(val_imgs):>3}")
print("✓ 8:2 분할 완료")


train : 120  |  val :  30
✓ 8:2 분할 완료


In [2]:
255**512

1407901858377921994410345688347596896619619142554097837765077305148540510077094158138835566039163737155746091366226562666854373712403671025943068839906270585035957311962185399524989371543048689004511684656386843101501008496414358411392170621564382768966260620621986470860550775751626384899641625777577457913603596025941526639675810071280201839834756161840602910202809542836802816749178368801585138736023604569890175093199392905841603855033866122594486065187626971582618480128073338895191481687490228032565931644555743022092116292228935633910516389275011109617569029043559531801263617794176928430086827401965727594462177163952633789657389842689868739374343166107474878193289257285976175466732066616970685313668103034234519951448917009923194202178864176156246959071282941365843865882419093367943264190686296306546548844741383683938429519234873384049527573382692107914230575208339320439604553200733151474600705968911913823461730687603077127284011706866998887268221895213478862301982395565993705836368849